# Data preparation section

In [1]:
# Data Wrangling
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
import calendar

# Vizualizacao
import seaborn as sns
import matplotlib.pyplot as plt

# To get clean data
import pickle

## Load clean data

In [15]:
df = pd.read_pickle('./data/clean_data.pkl')

In [16]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392677 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    392677 non-null  object 
 1   StockCode    392677 non-null  object 
 2   Description  392677 non-null  object 
 3   Quantity     392677 non-null  int64  
 4   InvoiceDate  392677 non-null  object 
 5   UnitPrice    392677 non-null  float64
 6   CustomerID   392677 non-null  string 
 7   Country      392677 non-null  object 
dtypes: float64(1), int64(1), object(5), string(1)
memory usage: 27.0+ MB


## Feature Engineering

### Add total price column

In [18]:
df['TotalPrice'] = df['Quantity']*df['UnitPrice']

In [19]:
df.reset_index(inplace=True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392677 entries, 0 to 392676
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   index        392677 non-null  int64  
 1   InvoiceNo    392677 non-null  object 
 2   StockCode    392677 non-null  object 
 3   Description  392677 non-null  object 
 4   Quantity     392677 non-null  int64  
 5   InvoiceDate  392677 non-null  object 
 6   UnitPrice    392677 non-null  float64
 7   CustomerID   392677 non-null  string 
 8   Country      392677 non-null  object 
 9   TotalPrice   392677 non-null  float64
dtypes: float64(2), int64(2), object(5), string(1)
memory usage: 30.0+ MB


### 'InvoiceDate' column to datetime

In [21]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%m/%d/%Y %H:%M')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392677 entries, 0 to 392676
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   index        392677 non-null  int64         
 1   InvoiceNo    392677 non-null  object        
 2   StockCode    392677 non-null  object        
 3   Description  392677 non-null  object        
 4   Quantity     392677 non-null  int64         
 5   InvoiceDate  392677 non-null  datetime64[ns]
 6   UnitPrice    392677 non-null  float64       
 7   CustomerID   392677 non-null  string        
 8   Country      392677 non-null  object        
 9   TotalPrice   392677 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4), string(1)
memory usage: 30.0+ MB


### Use only relevant columns for clustering (RFM model)

Only these columns are going to be used in clustering:

- CustomerID
- TotalPrice
- InvoiceDate (calculation of recency and frequency)
- InvoiceNo (to group by)

In [28]:
df.drop(columns = ['StockCode', 'Description', 'Quantity', 'UnitPrice', 'Country','index'], inplace= True)

In [23]:
# In this case, data range from 2010 - 2011. Using the last date in the dataset
last_date = max(df['InvoiceDate'])
print(last_date)

2011-12-09 12:50:00


In [39]:
# Get RFM dataset grouping by total price of sales
rfm = df.groupby(['CustomerID', 'InvoiceNo','InvoiceDate'])['TotalPrice'].agg('sum').reset_index()
rfm.head()

,CustomerID,InvoiceNo,InvoiceDate,TotalPrice
0,12347,537626,2010-12-07 14:57:00,711.79
1,12347,542237,2011-01-26 14:30:00,475.39
2,12347,549222,2011-04-07 10:43:00,636.25
3,12347,556201,2011-06-09 13:01:00,382.52
4,12347,562032,2011-08-02 08:48:00,584.91


### Recency

In [40]:
rfm['R (days)'] = last_date - rfm['InvoiceDate']
rfm.head()

,CustomerID,InvoiceNo,InvoiceDate,TotalPrice,R (days)
0,12347,537626,2010-12-07 14:57:00,711.79,366 days 21:53:00
1,12347,542237,2011-01-26 14:30:00,475.39,316 days 22:20:00
2,12347,549222,2011-04-07 10:43:00,636.25,246 days 02:07:00
3,12347,556201,2011-06-09 13:01:00,382.52,182 days 23:49:00
4,12347,562032,2011-08-02 08:48:00,584.91,129 days 04:02:00


### Frequency and Monetary

In [41]:
rfm = (
    rfm.groupby('CustomerID')
    .agg({'R (days)' : np.min, 'CustomerID': 'count', 'TotalPrice': 'mean'})
)

In [42]:
rfm['R (days)'] = rfm['R (days)'].apply(lambda x: x.days)

In [43]:
rfm['TotalPrice'] = rfm['TotalPrice'].apply(lambda x: "{:.2f}".format(x))
rfm.rename(columns = {'CustomerID':'F', 'TotalPrice': 'M (R$)'}, inplace = True)

In [44]:
rfm.head(10)

,R (days),F,M (R$)
CustomerID,,,
12347,1,7,615.71
12348,74,4,449.31
12349,18,1,1757.55
12350,309,1,334.40
12352,35,8,313.25
12353,203,1,89.00
12354,231,1,1079.40
12355,213,1,459.40
12356,22,3,937.14


## Save RFM dataset as pickle

In [45]:
rfm.to_pickle('./data/rfm.pkl')